In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'covid-pneumonia-normal-chest-xray-images:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1105311%2F1857760%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240802%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240802T072034Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2d02057dc4fe5f2c04178ae555326785cd936eda9957b7966172d6eb62c2a2118620a539cec4a67bb6790507010400da2e187a7d5ab53f289df49fca37963ad345a85a3ec1a4ef4151befa310a39d38fe3961cab2debe925049a3300e22766a7d39e6c947490ffe892f5437fa0790c7c42ade60a60b547b9162ea1b52b61d153a77049e1206649e8db03a07041a606051428299b2536523847ec109068078f3314ea73e464185ffc82232489effb01f6ce3eaed06ae93fe8d1da2866f4c436976c0130159b8e861aa2d219c1931c8b832e30afc67cd6c79626bf13f63d584a2fcc57b5bf93661b3c308af61dfbac50fac59e654bb2d1c4316be395739fe3b366'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 290862263 bytes downloaded
Downloaded and uncompressed: covid-pneumonia-normal-chest-xray-images
Data source import complete.


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.metrics import top_k_categorical_accuracy
import os
import random
import shutil
import re
from sklearn.model_selection import train_test_split

In [3]:
data='/kaggle/input/covid-pneumonia-normal-chest-xray-images'
output_dir = '/kaggle/working'
train_folder = os.path.join(output_dir, 'train')
val_folder = os.path.join(output_dir, 'val')
test_folder = os.path.join(output_dir, 'test')
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

In [4]:
def extract_label(filename):
    # Placeholder: Implement actual label extraction logic if needed
    return os.path.basename(os.path.dirname(filename))

In [5]:
imgs_list = []
for root, _, files in os.walk(data):
    for filename in files:
        if os.path.splitext(filename)[-1].lower() in image_extensions:
            img_path = os.path.join(root, filename)
            label = extract_label(img_path)
            imgs_list.append((img_path, label))

# Split data into training, validation, and testing sets
if imgs_list:  # Check if any images were found
    img_paths = [img_path for img_path, _ in imgs_list]
    labels = [label for _, label in imgs_list]

    X_train, X_test_val, y_train, y_test_val = train_test_split(
        img_paths,
        labels,
        test_size=0.2,
        stratify=labels,
        random_state=42
    )

    X_test, X_val, y_test, y_val = train_test_split(
        X_test_val, y_test_val, test_size=0.5, random_state=42
    )

In [6]:
def create_class_subdirectories_and_copy(base_folder, img_paths, labels):
    for img_path, label in zip(img_paths, labels):
        class_folder = os.path.join(base_folder, label)
        if not os.path.exists(class_folder):
            os.makedirs(class_folder)
        shutil.copy(img_path, os.path.join(class_folder, os.path.basename(img_path)))

# Copy images to the appropriate folders
create_class_subdirectories_and_copy(train_folder, X_train, y_train)
create_class_subdirectories_and_copy(val_folder, X_val, y_val)
create_class_subdirectories_and_copy(test_folder, X_test, y_test)

# Print file counts to verify
print(f"Train files: {len(X_train)}")
print(f"Validation files: {len(X_val)}")
print(f"Test files: {len(X_test)}")

Train files: 4182
Validation files: 523
Test files: 523


In [7]:
random.seed(42)
random.shuffle(imgs_list)

In [8]:
# Data Augumentation
datagen = ImageDataGenerator(
    rescale=1.0/255,
    samplewise_center=True,
    samplewise_std_normalization=True,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    cval=0,
    validation_split=0.2,
)

In [9]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
# Create training data generator
train_generator = datagen.flow_from_directory(
    train_folder,  # Use the correct path here
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='training',
    color_mode='rgb',
    shuffle=True,
    seed=2022
)

Found 3346 images belonging to 3 classes.


In [11]:
validation_generator = datagen.flow_from_directory(
    val_folder,  # Use the correct path here
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 103 images belonging to 3 classes.


In [12]:
# Initialize the test generator
test_generator = test_datagen.flow_from_directory(
    test_folder,  # Use the test folder path
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=False  # No need to shuffle the test data
)

Found 523 images belonging to 3 classes.


# **VGG19 Network**

In [13]:
from keras.applications import VGG19

In [15]:
!pip install tensorflow
!pip install keras

In [19]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
from glob import glob
from PIL import Image
import numpy as np
import pandas as pd
import cv2
import random
import albumentations as A
import keras
import matplotlib.cm as cm
import plotly.express as px
import plotly.figure_factory as ffis
import matplotlib.pyplot as plt
import seaborn as sns
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from keras.preprocessing import image
import tensorflow as tf
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, recall_score, accuracy_score, precision_score, f1_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, Concatenate, GlobalAveragePooling2D, Dense, Multiply, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
#from keras.utils import layer_utils
from tensorflow.python.keras.utils import layer_utils
#from keras.utils.data_utils import get_file
from tensorflow.python.keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
#from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.utils import plot_model
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

In [20]:
base_model = VGG19(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)
modelV = Model(inputs=base_model.input, outputs=predictions)

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [22]:
for layer in base_model.layers:
    layer.trainable = False

modelV.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=20,          # Number of epochs with no improvement to wait
    restore_best_weights=True,  # Restore the best model weights when stopping
)

In [ ]:
modelV.fit(train_generator, epochs=20, validation_data=validation_generator, callbacks=[early_stopping])

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


In [ ]:
test_loss_v, test_accuracy_v = modelV.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test accuracy: {test_accuracy_v}")

# **Inception Network**

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(input_shape = (150, 150, 3), include_top = False, weights = 'imagenet')

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(1, activation='softmax')(x)

modelI = tf.keras.models.Model(base_model.input, x)

modelI.compile(optimizer = RMSprop(lr=0.0001), loss = 'MSE', metrics = ['acc'])

In [ ]:
modelI.fit(train_generator, epochs=20, validation_data=validation_generator, callbacks=[early_stopping])

In [ ]:
test_loss_i, test_accuracy_i = modelI.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test accuracy: {test_accuracy_i}")

# **Comparision and Evaluation of both the networks and their results**

In [ ]:
print('VGG19 Model')
modelV.summary()
print('InceptionV3')
modelI.summary()